## Parse txt file

In [153]:
years = {str(i) for i in range(1900, 2024)}.union({'S/A'})

In [154]:
# read raw content
with open('v1-cleaned.txt', 'r') as f:
    content_raw = f.read()
content_raw[:100]

'\n\n\n\n\nMEZZE BOTTIGLIE\n\nCHAMPAGNE E SPUMANTI\n\nS/A\tRUINART BLANC DE BLANCS\tRUINART\n\t\t\t€ 90,00\n\nS/A\tRUIN'

In [155]:
# remove multiple spaces, tabs and newlines

import re
content = re.sub(r'\n+', '\n', content_raw)
content = re.sub(r'  +', '\t', content)
content = re.sub(r'\(', '\t', content)
content = re.sub(r'\)', '\t', content)
content = re.sub(r'\t+', '\t', content)
content[:100]

"\nMEZZE BOTTIGLIE\nCHAMPAGNE E SPUMANTI\nS/A\tRUINART BLANC DE BLANCS\tRUINART\n\t€ 90,00\nS/A\tRUINART ROSE'"

In [156]:
# split content into lines

content_lines = content.split('\n')

print(len(content_lines), "content lines")
display(content_lines[69:80])

# drop header lines
print("Header lines to drop:")
for i, line in enumerate(content_lines):
    if "€" not in line and "\t" not in line and line not in years:
        print(i, line)

content_lines = [line for line in content_lines if "€" in line or "\t" in line or line in years]
content_lines[69:90]

1854 content lines


['2002\tFratelli Lunelli\t€ 550,00',
 '2008\tGIULIO FERRARI RIS. DEL FONDATORE\tFERRARI',
 '2010\tFratelli Lunelli\t€ 240,00',
 '2007\tGIULIO FERRARI RIS. DEL FONDATORE\tMagnum\tFERRARI',
 '2008\tFratelli Lunelli\t€ 480,00',
 '2009',
 'SPUMANTI\tITALIANI',
 '2018\tBRUT\tKETTMEIER',
 '\t€ 42,00',
 "2010\tBRUT METODO CLASSICO ''TOMBOLA DI PIN''\tDE STEFANI",
 '\t€ 50,00']

Header lines to drop:
0 
1 MEZZE BOTTIGLIE
2 CHAMPAGNE E SPUMANTI
17 BIANCHI
34 ROSSI
49 SPUMANTI E CHAMPAGNE
50 ITALIANI
51 TRENTO DOC
98 IL FRANCIACORTA
131 IL PROSECCO
142 CHAMPAGNE
143 MONTAGNE DE REIMS
232 COTE DES BAR
235 COTE DES BLANCS
332 VALLE DE LA MARNE
413 VINI BIANCHI
414 VENETO
433 ALTO ADIGE - TRENTINO
497 FRIULI VENEZIA GIULIA
564 ABRUZZO
580 CAMPANIA
592 LOMBARDIA
597 MARCHE
614 PIEMONTE
627 PUGLIA
632 SARDEGNA
637 SICILIA
646 TOSCANA
661 UMBRIA
675 FRANCIA
676 ALSACE
685 BOURGOGNE
694 CHABLIS
697 CORTON CHARLEMAGNE
709 LADOIX-SERRIGNY
714 MEURSAULT
748 LE MONTRACHET
786 JURA
791 JURANCON
794 LANGUEDOC-ROUSSILLON
803 VALLE DELLA LOIRA
817 AUSTRIA
845 GERMANIA
846 MOSEL-SAAR-RUWER
867 NAHE
876 RHEINGAU
889 RHINEHESSEN
892 PFALZ
901 SLOVENIA
914 SPAGNA
919 VINI ROSSI
920 AMARONE E VALPOLICELLA
921 COMUNE DI FUMANE
937 COMUNE DI POIANO
942 COMUNE DI GREZZANA
945 COMUNE DI NEGRAR
950 COMUNE DI SAN PIETRO IN CARIANO
953 COMUNE DI ILLASI
960 COMUNE DI MARANO DI VALPOLICELLA

['\t€ 42,00',
 "2010\tBRUT METODO CLASSICO ''TOMBOLA DI PIN''\tDE STEFANI",
 '\t€ 50,00',
 'S/A\tBLANC DE BLANC\tMETODO CLASSICO EXTRA BRUT\tVILLA PARENS',
 '\t€ 40,00',
 '2013\tBRUT DURELLO \tBELLAGUARDIA',
 '\t€ 45,00',
 "2016\tBRUT ROSE'\tBELLAGUARDIA",
 '\t€ 58,00',
 "S/A\tDURELLO PAS DOSE'\tDAL MASO",
 '\t€ ,00',
 "S/A\tEXTRA BRUT CUVEE' MARIANNA\tARUNDA\t",
 '\t€ 68,00',
 '2016\tLEONIA POMINO BRUT\tFRESCOBALDI',
 '\t€ 45,00',
 "2014\tMATT '55\tPIAN DELLE VETTE",
 '\t€ 65,00',
 '2012\tPLUK BLANC DE NOIR BRUT NATURE\tBELLENDA',
 '\t€ 56,00',
 '2008\tWURM BLANC DE BLANC BRUT NATURE\tBELLENDA',
 '\t€ 60,00']

In [157]:
content_lines[65]

'2008\tFratelli Lunelli\t€ 480,00'

In [158]:
# join lines that belong to the same wine

"""This code is ugly, but it works.
The issue is that the lines that belong to the same wine are not always consecutive.

"""

wine_lines = []
wine_line = ''

i = 0
while i<len(content_lines):
    # ignore lines that do not start with a year
    if not content_lines[i].split('\t')[0].strip() in years:
        i+=1
        continue

    # join lines that belong to the same wine, until you find a "€" sign
    wine_line = content_lines[i]
    i+=1
    while "€" not in content_lines[i] and i<len(content_lines):
        wine_line += '\t' + content_lines[i]
        i+=1
    if i>=len(content_lines): # avoid out of bounds
        break

    wine_line += '\t' + content_lines[i] # add the line with the "€" sign
    i+=1
    if i>=len(content_lines): # avoid out of bounds
        break

    # add additional years to the wine line
    while content_lines[i].strip() in years and i<len(content_lines):
        wine_line += '\t' + content_lines[i]
        i+=1

    wine_lines.append(re.sub(r'\t+', '\t', wine_line))
    wine_line = ''

wine_lines[:40]

['S/A\tRUINART BLANC DE BLANCS\tRUINART\t€ 90,00',
 "S/A\tRUINART ROSE'\tRUINART\t€ 90,00",
 'S/A\tEXTRA BRUT PREMIERE CUVEE\tBRUNO PAILLARD\t€ 65,00',
 'S/A\tBRUT CHAMPAGNE\tSPECIAL CUVEE\tBOLLINGER\t€ 75,00',
 'S/A\tBRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS\tPERTOIS MORISET\t€ 60,00',
 'S/A\tBRUT ROEDERER COLLECTION 244\tLOUIS ROEDERER\tLouis Roederer\t€ 65,00',
 "S/A\tEXTRA BRUT CUVEE' MARIANNA\tARUNDA\t€ 36,00",
 '2020\tCORTON CHARLEMAGNE \t BONNEAU DU MARTRAY\tGrand Cru\t€ ,00',
 '2021\tCHAMPAGNIS CHARDONNAY \t VIE DE ROMANS\t€ 30,00',
 '2021\tCERVARO DELLA SALA\t ANTINORI\t2022\t€ 65,00',
 '2021\tDESSIMIS PINOT GRIGIO\tVIE DE ROMANS\t€ 30,00',
 '2022\tSAUVIGNON\tCANTINA S. MICHELE APPIANO\t€ 30,00',
 '2022\tSAUVIGNON LAFOA\tCOLTERENZIO\t€ 35,00',
 '2021\tVIERIS SAUVIGNON\tVIE DE ROMANS\t€ 30,00',
 '2018\tGAJA E REY\tGAJA\t€ 240,00',
 '2018\tAMARONE\tALLEGRINI\tdoc della Valpolicella\t€ 75,00',
 '2020\tCHIANTI CLASSICO\tFONTODI\t€ 24,00',
 '2021\tPINOT NERO PATRICIA\tGIRLAN\t€ 26,

## Create clean wine info

In [159]:
import pandas as pd

In [160]:
formats = {'Magnum', 'Doppio Magnum', '6L', '6l', 'Piccolo'}

In [161]:
for wine_line in wine_lines:
    if any(format.lower() in wine_line.lower() for format in formats):
        # print(wine_line)
        pass

wine_info = []

for line in wine_lines:
    naming = [l.strip() for l in line.split('\t') if l not in years and l not in formats and '€' not in l]
    naming = [n for n in naming if n]
    format = [l.strip() for l in line.split('\t') if l in formats]
    format = format[0] if format else 'standard'
    vintages = [l.strip() for l in line.split('\t') if l in years]
    price = [l.strip() for l in line.split('\t') if '€' in l]

    wine_info.append({
        'naming': naming,
        'format': format,
        'years': vintages,
        'price': price
    })

wine_info[:5]

[{'naming': ['RUINART BLANC DE BLANCS', 'RUINART'],
  'format': 'standard',
  'years': ['S/A'],
  'price': ['€ 90,00']},
 {'naming': ["RUINART ROSE'", 'RUINART'],
  'format': 'standard',
  'years': ['S/A'],
  'price': ['€ 90,00']},
 {'naming': ['EXTRA BRUT PREMIERE CUVEE', 'BRUNO PAILLARD'],
  'format': 'standard',
  'years': ['S/A'],
  'price': ['€ 65,00']},
 {'naming': ['BRUT CHAMPAGNE', 'SPECIAL CUVEE', 'BOLLINGER'],
  'format': 'standard',
  'years': ['S/A'],
  'price': ['€ 75,00']},
 {'naming': ['BRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS', 'PERTOIS MORISET'],
  'format': 'standard',
  'years': ['S/A'],
  'price': ['€ 60,00']}]

In [162]:
# sanity check on price
for i, info in enumerate(wine_info):
    if len(info['price']) > 1:
        print(i, info)

# Oh no! There are some lines with 2 wines! Let's split them manually
# The reason is that there is no new line between the wines
# I solve this in the most basic way, by correcting v1-cleaned.txt
# So after the correction here there will be nothing

In [163]:
# sanity check on naming: length equal to 3
n = 0
for i, info in enumerate(wine_info):
    if len(info['naming']) == 3:
        n+=1
        print(i, info['naming'])
print(n)

3 ['BRUT CHAMPAGNE', 'SPECIAL CUVEE', 'BOLLINGER']
5 ['BRUT ROEDERER COLLECTION 244', 'LOUIS ROEDERER', 'Louis Roederer']
7 ['CORTON CHARLEMAGNE', 'BONNEAU DU MARTRAY', 'Grand Cru']
15 ['AMARONE', 'ALLEGRINI', 'doc della Valpolicella']
19 ['ORNELLAIA', "TENUTA DELL'ORNELLAIA", 'Bolgheri Superiore']
20 ['RIPASSA', 'ZENATO', 'doc della Valpolicella']
21 ['SASSICAIA', 'TENUTA SAN GUIDO', 'doc Bolgheri']
22 ['ALTEMASI', 'CAVIT', 'Trento Doc']
23 ['OPERA BRUT', 'CORVEE', 'Trento Doc']
24 ["BRUT FERRARI PERLE'", 'FERRARI', 'Fratelli Lunelli']
25 ["BRUT FERRARI PERLE' ROSE'", 'FERRARI', 'Fratelli Lunelli']
26 ["BRUT FERRARI PERLE' ROSE'", 'FERRARI', 'Fratelli Lunelli']
27 ["BRUT FERRARI PERLE' BIANCO", 'FERRARI', 'Fratelli Lunelli']
28 ["BRUT FERRARI PERLE' CUVEE ZERO", 'FERRARI', 'Fratelli Lunelli']
29 ["BRUT FERRARI PERLE' NERO", 'FERRARI', 'Fratelli Lunelli']
30 ['GIULIO FERRARI RIS. DEL FONDATORE COLLEZIONE', 'FERRARI', 'Fratelli Lunelli']
31 ['GIULIO FERRARI RIS. DEL FONDATORE', 'FERRARI

In [164]:
# sanity check on naming: length greater or equal to 4
for i, info in enumerate(wine_info):
    if len(info['naming']) >= 4:
        print(i, info['naming'])

45 ['BRUT FRANCIACORTA ALMA', 'GRANDE CUVEE', 'BELLAVISTA', 'docg Franciacorta']
47 ['BRUT FRANCIACORTA ALMA', 'GRANDE CUVEE', 'BELLAVISTA', 'docg Franciacorta']
48 ['BRUT FRANCIACORTA ALMA', 'GRANDE CUVEE', 'BELLAVISTA', 'docg Franciacorta']
60 ['PROSECCO', 'EXTRA DRY', 'TORRESELLA', 'Prosecco Doc']
95 ['CRISTAL', 'BRUT', 'LOUIS ROEDERER', 'Louis Roederer']
96 ['CRISTAL', 'BRUT', 'LOUIS ROEDERER', 'Louis Roederer']
97 ['CRISTAL', 'BRUT', 'LOUIS ROEDERER', 'Louis Roederer']
98 ['CRISTAL', 'BRUT', 'ROSE’', 'LOUIS ROEDERER', 'Louis Roederer']
263 ['SAUVIGNON', 'RUTTARS', 'VILLA PARENS', 'Venezia Giulia Igp']
280 ['RADICI', 'FIANO DI AVELLINO RIS.', 'MASTROBERARDINO', 'Docg']
308 ['ORNELLAIA', 'BIANCO', "TENUTA DELL'ORNELLAIA", 'Sauvignon Blanc e Viognier']
309 ['ORNELLAIA', 'BIANCO', "TENUTA DELL'ORNELLAIA", 'Sauvignon Blanc e Viognier']
359 ['PULIGNY-MONTRACHET 1er CRU', 'LES FOLATIERES', 'a. Puligny Montrachet c.', 'DOMAINE ALAIN CHAVY']
366 ['PULIGNY-MONTRACHET 1er CRU', 'SOUS LES PUI

In [165]:
# sanity check on naming: price is in the name
for i, info in enumerate(wine_info):
    if any(character.isdigit() for character in info['naming'][-1]):
        print(i, info)

# correct naming: 20072000022
# some have the price in the name, because there are 2 spaces between € and the price
# i correct this in v1-cleaned.txt, so they do not appear here
# the remaining ones are fine


717 {'naming': ['CHATEAU HAUT BRION', 'CHATEAU HAUT BRION', '1er Grand Cru Classe'], 'format': 'standard', 'years': ['2019'], 'price': ['€ 1.000,00']}
718 {'naming': ['CHATEAU HAUT BRION', 'CHATEAU HAUT BRION', '1er Grand Cru Classe'], 'format': 'standard', 'years': ['2000'], 'price': ['€ 1.780,00']}
719 {'naming': ['CHATEAU PONTET CANET', 'CHATEAU PONTET CANET', "5eme Cru Classe'"], 'format': 'standard', 'years': ['2020'], 'price': ['€ 200,00']}
722 {'naming': ['CHATEAU HAUT-BATAILLEY', 'CHATEAU HAUT BATAILLEY', "5eme Cru Classe'"], 'format': 'standard', 'years': ['2016'], 'price': ['€ 180,00']}
723 {'naming': ['CHATEAU LAFITE ROTHSCHILD', 'DOMAINE BARONS DE ROTHSCHILD', '1er Cru Classe'], 'format': 'standard', 'years': ['2000'], 'price': ['€ 2.200,00']}
724 {'naming': ['CHATEAU LATOUR', 'DOMAINE LATOUR', '1er Grand Cru Classe'], 'format': 'standard', 'years': ['2000'], 'price': ['€ 1.800,00']}
725 {'naming': ['CHATEAU LAFITE ROTHSCHILD', 'DOMAINE BARONS DE ROTHSCHILD', '1er Cru Class

# Create v2-cleaned.csv

In [166]:
for info in wine_info:
    if len(info['naming']) == 2:
        info['name'] = info['naming'][0]
        info['winery_name'] = info['naming'][1]
    elif len(info['naming']) == 3:
        info['name'] = info['naming'][0]
        info['winery_name'] = info['naming'][1]
        info['denomination'] = info['naming'][2]
    else:
        info['name'] = info['naming'][0] + ' ' + info['naming'][1]
        info['winery_name'] = info['naming'][2]
        info['denomination'] = info['naming'][3]

In [167]:
import copy

wines = pd.DataFrame(wine_info)
wines = wines.drop('naming', axis=1)
wines = wines.explode('years')
wines = wines.explode('price')

wines_original = copy.deepcopy(wines)

wines

,format,years,price,name,winery_name,denomination
0,standard,S/A,"€ 90,00",RUINART BLANC DE BLANCS,RUINART,NaN
1,standard,S/A,"€ 90,00",RUINART ROSE',RUINART,NaN
2,standard,S/A,"€ 65,00",EXTRA BRUT PREMIERE CUVEE,BRUNO PAILLARD,NaN
3,standard,S/A,"€ 75,00",BRUT CHAMPAGNE,SPECIAL CUVEE,BOLLINGER
4,standard,S/A,"€ 60,00",BRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS,PERTOIS MORISET,NaN
...,...,...,...,...,...,...
839,standard,S/A,"€ ,00",SHERRY PX PEDRO XIMENEZ,FERNANDO DE CASTILLA,NaN
840,standard,S/A,"€ 40,00",PORTO RESERVA ADRIANO,RAMOS PINTO,NaN
841,standard,2008,"€ 140,00",TOKAJI ASZU 5 PUTTONIOS,SAMUEL TINON,NaN
842,standard,2016,"€ 90,00",TOKAJI ASZU 3 PUTTONIOS,OREMUS,NaN


## Create standard fields

required fields:

 - `external_id`: missing
 - `name`: OK
 - `winery_name`: OK
 - `type`: missing
 - `storage_area`: missing
 - `size`: create from `format`
 - `vintage`: create from `years`
 - `price`: to parse
 - `info`: missing
 - `quantity`: missing
 - `internal_notes`: missing

In [168]:
# - `external_id`: missing
wines['external_id'] = wines.index + 1
wines.head()

,format,years,price,name,winery_name,denomination,external_id
0,standard,S/A,"€ 90,00",RUINART BLANC DE BLANCS,RUINART,NaN,1
1,standard,S/A,"€ 90,00",RUINART ROSE',RUINART,NaN,2
2,standard,S/A,"€ 65,00",EXTRA BRUT PREMIERE CUVEE,BRUNO PAILLARD,NaN,3
3,standard,S/A,"€ 75,00",BRUT CHAMPAGNE,SPECIAL CUVEE,BOLLINGER,4
4,standard,S/A,"€ 60,00",BRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS,PERTOIS MORISET,NaN,5


In [169]:
#  - `name`: OK
#  - `winery_name`: OK
#  - `type`: missing
wines['type'] = ''
wines.head()

,format,years,price,name,winery_name,denomination,external_id,type
0,standard,S/A,"€ 90,00",RUINART BLANC DE BLANCS,RUINART,NaN,1,
1,standard,S/A,"€ 90,00",RUINART ROSE',RUINART,NaN,2,
2,standard,S/A,"€ 65,00",EXTRA BRUT PREMIERE CUVEE,BRUNO PAILLARD,NaN,3,
3,standard,S/A,"€ 75,00",BRUT CHAMPAGNE,SPECIAL CUVEE,BOLLINGER,4,
4,standard,S/A,"€ 60,00",BRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS,PERTOIS MORISET,NaN,5,


In [170]:
#  - `storage_area`: missing
wines['storage_area'] = ''
wines.head()

,format,years,price,name,winery_name,denomination,external_id,type,storage_area
0,standard,S/A,"€ 90,00",RUINART BLANC DE BLANCS,RUINART,NaN,1,,
1,standard,S/A,"€ 90,00",RUINART ROSE',RUINART,NaN,2,,
2,standard,S/A,"€ 65,00",EXTRA BRUT PREMIERE CUVEE,BRUNO PAILLARD,NaN,3,,
3,standard,S/A,"€ 75,00",BRUT CHAMPAGNE,SPECIAL CUVEE,BOLLINGER,4,,
4,standard,S/A,"€ 60,00",BRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS,PERTOIS MORISET,NaN,5,,


In [171]:
wines['format'].unique()

array(['standard', 'Magnum', 'Doppio Magnum', '6L', 'Piccolo'],
      dtype=object)

In [172]:
#  - `size`: create from `format`

# map automatically
size_mapping = {
    'Piccolo': 'HALF_BOTTLE',
    'standard': 'BOTTLE',
    'Magnum': 'MAGNUM',
    'Doppio Magnum': 'JEROBOAM',
    '6L': 'MATHUSALEM',
}
wines['size'] = wines['format'].map(size_mapping)

# map manually the first ones
wines.loc[range(0, 7), 'size'] = 'HALF_BOTTLE'
wines.head(8)

,format,years,price,name,winery_name,denomination,external_id,type,storage_area,size
0,standard,S/A,"€ 90,00",RUINART BLANC DE BLANCS,RUINART,NaN,1,,,HALF_BOTTLE
1,standard,S/A,"€ 90,00",RUINART ROSE',RUINART,NaN,2,,,HALF_BOTTLE
2,standard,S/A,"€ 65,00",EXTRA BRUT PREMIERE CUVEE,BRUNO PAILLARD,NaN,3,,,HALF_BOTTLE
3,standard,S/A,"€ 75,00",BRUT CHAMPAGNE,SPECIAL CUVEE,BOLLINGER,4,,,HALF_BOTTLE
4,standard,S/A,"€ 60,00",BRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS,PERTOIS MORISET,NaN,5,,,HALF_BOTTLE
5,standard,S/A,"€ 65,00",BRUT ROEDERER COLLECTION 244,LOUIS ROEDERER,Louis Roederer,6,,,HALF_BOTTLE
6,standard,S/A,"€ 36,00",EXTRA BRUT CUVEE' MARIANNA,ARUNDA,NaN,7,,,HALF_BOTTLE
7,standard,2020,"€ ,00",CORTON CHARLEMAGNE,BONNEAU DU MARTRAY,Grand Cru,8,,,BOTTLE


In [173]:
#  - `vintage`: create from `years`
wines['vintage'] = wines['years']

In [186]:
#  - `price`: to parse
import re

def parse_price(price):
    orginal_price = price
    try:
        price = price.replace('.','').replace(',','.')
        price = price.replace('€', '').replace(' ', '').strip()
        price = re.sub(r'€\s*', '', price)
        price = re.sub(r'^\W+|\W+$', '', price)
        return float(price if price else 0)
    except Exception as e:
        print(f"Error parsing price: {orginal_price}")
        return orginal_price

print('applying parse_price')
wines['price'] = wines_original['price'].apply(parse_price)

print('\nmanually correcting prices that were not parsed correctly:')
wines.loc[wines_original['price'] == '€ 65,00 ,00', 'price'] = 65.0
wines.loc[wines_original['price'] == '€ 140,00 ,00', 'price'] = 140.0
print('done.')

wines['price'] = wines['price'].astype(float)
wines.head()

applying parse_price
Error parsing price: € 65,00 ,00
Error parsing price: € 140,00 ,00

manually correcting prices that were not parsed correctly:
done.


,format,years,price,name,winery_name,denomination,external_id,type,storage_area,size,vintage
0,standard,S/A,90.0,RUINART BLANC DE BLANCS,RUINART,NaN,1,,,HALF_BOTTLE,S/A
1,standard,S/A,90.0,RUINART ROSE',RUINART,NaN,2,,,HALF_BOTTLE,S/A
2,standard,S/A,65.0,EXTRA BRUT PREMIERE CUVEE,BRUNO PAILLARD,NaN,3,,,HALF_BOTTLE,S/A
3,standard,S/A,75.0,BRUT CHAMPAGNE,SPECIAL CUVEE,BOLLINGER,4,,,HALF_BOTTLE,S/A
4,standard,S/A,60.0,BRUT CHAMP. GRAND CRU B&B QUATRE TERROIRS,PERTOIS MORISET,NaN,5,,,HALF_BOTTLE,S/A


In [187]:
#  - `info`: missing
wines['info'] = ''

In [188]:
#  - `quantity`: missing
wines['quantity'] = 0

In [189]:
#  - `internal_notes`: missing
wines['internal_notes'] = ''

## Create v2 files

In [192]:
wines_out = wines[['external_id', 'name', 'winery_name', 'type', 'storage_area', 'size', 'vintage', 'price', 'info', 'quantity', 'internal_notes']]

open('v2-cleaned.csv', 'w').close()
wines_out.to_csv('v2-cleaned.csv', index=False)


In [193]:
with open('search-terms.txt', 'w') as f:
    for term in wines['name'].unique():
        f.write(f'{term}\n')
    for term in wines['winery_name'].unique():
        f.write(f'{term}\n')